# Sample sequences from our full ETL load

- subselect sequences
- subselect columns
- remove specimens that violate some constraints: too few sequences, or not all isotypes found

both peak + off-peak are still included after this.

In [1]:
from malid import config, helpers
from malid.sample_sequences import sample_sequences

**If regenerating, clear `config.paths.sequences_sampled` first with `rm -r`**

In [2]:
config.paths.sequences_sampled

PosixPath('/users/maximz/code/boyd-immune-repertoire-classification/data/data_v_20221224/sequences.sampled.parquet')

In [3]:
import dask
import dask.dataframe as dd
import time

In [4]:
from dask.distributed import Client

# multi-processing backend
# access dashbaord at http://127.0.0.1:61083
client = Client(
    scheduler_port=61084,
    dashboard_address=":61083",
    n_workers=4,
    processes=True,
    threads_per_worker=8,
    memory_limit="125GB",  # per worker
)
display(client)
# for debugging: client.restart()

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:61083/status,
Dashboard: http://127.0.0.1:61083/status,Workers: 4
Total threads: 32,Total memory: 465.66 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:61084,Workers: 4
Dashboard: http://127.0.0.1:61083/status,Total threads: 32
Started: Just now,Total memory: 465.66 GiB
Comm: tcp://127.0.0.1:33435,Total threads: 8
Dashboard: http://127.0.0.1:33061/status,Memory: 116.42 GiB
Nanny: tcp://127.0.0.1:35009,


In [5]:
# Input columns
desired_columns = [
    "specimen_label",
    "participant_label",
    "v_gene",
    "j_gene",
    "disease",
    "disease_subtype",
    "cdr1_seq_aa_q_trim",
    "cdr2_seq_aa_q_trim",
    "cdr3_seq_aa_q_trim",
    "cdr3_aa_sequence_trim_len",
    "extracted_isotype",
    "isotype_supergroup",
    "v_mut",
    "num_reads",
    "igh_or_tcrb_clone_id",
]

In [6]:
# Don't use fastparquet, because it changes specimen labels like M54-049 to 2049-01-01 00:00:54 -- i.e. it coerces partition names to numbers or dates
df = dd.read_parquet(config.paths.sequences, columns=desired_columns, engine="pyarrow")

In [7]:
# each partition is a specimen
df.npartitions

717

In [8]:
df

,specimen_label,participant_label,v_gene,j_gene,disease,disease_subtype,cdr1_seq_aa_q_trim,cdr2_seq_aa_q_trim,cdr3_seq_aa_q_trim,cdr3_aa_sequence_trim_len,extracted_isotype,isotype_supergroup,v_mut,num_reads,igh_or_tcrb_clone_id
npartitions=717,,,,,,,,,,,,,,,
,category[known],category[known],category[unknown],category[unknown],category[unknown],category[unknown],object,object,object,int64,category[unknown],category[unknown],float64,int64,int64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [9]:
# required_gene_loci=config.gene_loci_used
# Required gene loci may differ for each specimen. Prepare a dict
required_gene_loci = helpers._load_etl_metadata()["available_gene_loci"]
required_gene_loci

specimen_label
M418-S001    (((GeneLocus.BCR)), ((GeneLocus.TCR)))
M418-S007    (((GeneLocus.BCR)), ((GeneLocus.TCR)))
M418-S008    (((GeneLocus.BCR)), ((GeneLocus.TCR)))
M418-S009    (((GeneLocus.BCR)), ((GeneLocus.TCR)))
M418-S010    (((GeneLocus.BCR)), ((GeneLocus.TCR)))
                              ...                  
M464-S042    (((GeneLocus.BCR)), ((GeneLocus.TCR)))
M464-S043    (((GeneLocus.BCR)), ((GeneLocus.TCR)))
M464-S044    (((GeneLocus.BCR)), ((GeneLocus.TCR)))
M464-S045    (((GeneLocus.BCR)), ((GeneLocus.TCR)))
M464-S046    (((GeneLocus.BCR)), ((GeneLocus.TCR)))
Name: available_gene_loci, Length: 717, dtype: object

In [10]:
required_gene_loci.value_counts()

(((GeneLocus.BCR)), ((GeneLocus.TCR)))    634
(((GeneLocus.BCR)))                        83
Name: available_gene_loci, dtype: int64

In [11]:
# pass empty df as meta, along with the new columns created by sample_sequences
meta = df.head(0).assign(total_clone_num_reads=0, num_clone_members=0)
df_sampled = df.map_partitions(
    sample_sequences, required_gene_loci=required_gene_loci.to_dict(), meta=meta
)
df_sampled

,specimen_label,participant_label,v_gene,j_gene,disease,disease_subtype,cdr1_seq_aa_q_trim,cdr2_seq_aa_q_trim,cdr3_seq_aa_q_trim,cdr3_aa_sequence_trim_len,extracted_isotype,isotype_supergroup,v_mut,num_reads,igh_or_tcrb_clone_id,total_clone_num_reads,num_clone_members
npartitions=717,,,,,,,,,,,,,,,,,
,category[known],category[known],category[known],category[known],category[known],category[known],object,object,object,int64,category[known],category[known],float64,int64,int64,int64,int64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [12]:
itime = time.time()

# This can behave weirdly with empty partitions. https://github.com/dask/dask/issues/8832

df_sampled.to_parquet(
    config.paths.sequences_sampled,
    overwrite=True,
    compression="snappy",  # gzip
    engine="pyarrow",
    # schema arg only accepted by pyarrow engine:
    # Set schema to "infer" if we have any empty partitions and using pyarrow.
    # schema="infer" is no longer slow as of https://github.com/dask/dask/pull/9131
    # schema=None breaks downstream readers.
    schema="infer",
    # also, do empty partitions even make it to disk, or are they eliminated? they seem eliminated.
    write_metadata_file=False,
    partition_on=["participant_label", "specimen_label"],
)

print(time.time() - itime)

2023-01-14 03:27:49,896 - malid.sample_sequences - INFO - Participant BFI-0000234 specimen M132-S014 is empty


2023-01-14 03:27:53,930 - malid.sample_sequences - INFO - Participant BFI-0003708 specimen M132-S009 is empty


2023-01-14 03:27:54,285 - malid.sample_sequences - INFO - Participant BFI-0003709 specimen M132-S010 is empty
2023-01-14 03:27:54,414 - malid.sample_sequences - INFO - Participant BFI-0003707 specimen M132-S008 is empty


2023-01-14 03:27:55,094 - malid.sample_sequences - INFO - Participant BFI-0003710 specimen M132-S011 is empty
2023-01-14 03:27:55,122 - malid.sample_sequences - INFO - Participant BFI-0003711 specimen M132-S013 is empty


2023-01-14 03:27:58,423 - malid.sample_sequences - INFO - Participant BFI-0002859 specimen M114-S034 is empty


...

2023-01-14 03:45:20,821 - malid.sample_sequences - INFO - Removing BFI-0003069 specimen M64-020 because it did not have enough clones. Clone count by isotype: {'IGHG': 42, 'IGHA': 1419, 'IGHD-M': 24388, 'TCRB': 102040}


2023-01-14 03:45:23,044 - malid.sample_sequences - INFO - Removing BFI-0003073 specimen M64-024 because it did not have enough clones. Clone count by isotype: {'IGHG': 24, 'IGHA': 24, 'IGHD-M': 53, 'TCRB': 1406}
2023-01-14 03:45:23,049 - malid.sample_sequences - INFO - Removing BFI-0003074 specimen M64-025 because it did not have enough clones. Clone count by isotype: {'IGHG': 33, 'IGHA': 17, 'IGHD-M': 65, 'TCRB': 1003}


2023-01-14 03:45:32,435 - malid.sample_sequences - INFO - Removing BFI-0003094 specimen M64-045 because it did not have enough clones. Clone count by isotype: {'IGHG': 3808, 'IGHA': 25, 'IGHD-M': 99002, 'TCRB': 77262}


2023-01-14 03:45:54,488 - malid.sample_sequences - INFO - Participant BFI-0000258 specimen M114-S055 is empty


1149.2107574939728


In [13]:
df_sampled.dtypes

specimen_label               category
participant_label            category
v_gene                       category
j_gene                       category
disease                      category
disease_subtype              category
cdr1_seq_aa_q_trim             object
cdr2_seq_aa_q_trim             object
cdr3_seq_aa_q_trim             object
cdr3_aa_sequence_trim_len       int64
extracted_isotype            category
isotype_supergroup           category
v_mut                         float64
num_reads                       int64
igh_or_tcrb_clone_id            int64
total_clone_num_reads           int64
num_clone_members               int64
dtype: object

In [14]:
df_sampled2 = dd.read_parquet(config.paths.sequences_sampled, engine="pyarrow")
df_sampled2

,v_gene,j_gene,disease,disease_subtype,cdr1_seq_aa_q_trim,cdr2_seq_aa_q_trim,cdr3_seq_aa_q_trim,cdr3_aa_sequence_trim_len,extracted_isotype,isotype_supergroup,v_mut,num_reads,igh_or_tcrb_clone_id,total_clone_num_reads,num_clone_members,participant_label,specimen_label
npartitions=522,,,,,,,,,,,,,,,,,
,category[unknown],category[unknown],category[unknown],category[unknown],object,object,object,int64,category[unknown],category[unknown],float64,int64,int64,int64,int64,category[known],category[known]
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [15]:
# check dtypes
df_sampled2.dtypes

v_gene                       category
j_gene                       category
disease                      category
disease_subtype              category
cdr1_seq_aa_q_trim             object
cdr2_seq_aa_q_trim             object
cdr3_seq_aa_q_trim             object
cdr3_aa_sequence_trim_len       int64
extracted_isotype            category
isotype_supergroup           category
v_mut                         float64
num_reads                       int64
igh_or_tcrb_clone_id            int64
total_clone_num_reads           int64
num_clone_members               int64
participant_label            category
specimen_label               category
dtype: object

In [16]:
# expected lower because losing some empty specimens
df.npartitions, df_sampled.npartitions, df_sampled2.npartitions

(717, 717, 522)

In [17]:
client.shutdown()